In [ ]:
import json
import sys
import pandas as pd
import numpy as np
import plotly.express as px
# import pandas as pd
from datetime import datetime
from tronscan.client import TronscanClient

sys.path.append('..')

from env import settings

%load_ext autoreload
%autoreload 2

In [ ]:
client = TronscanClient(api_key=settings.TRONSCAN_API_TOKEN)
my_addr = settings.TRON_ADDRS[0]
my_addr_2 = settings.TRON_ADDRS[1]

In [ ]:
transfers = list(client.all_transfers(my_addr)) + list(client.all_transfers(my_addr_2))

In [ ]:
len(transfers)

In [ ]:
transfers[0]

In [ ]:
usdt_transfers = [
    ((1 if t['to_address'] == my_addr else -1) * float(int(t['quant']) / 10**t['tokenInfo']['tokenDecimal']), datetime.fromtimestamp(int(t['block_ts']) / 1000))
    for t in transfers if t['tokenInfo']['tokenId'] == 'TR7NHqjeKQxGTCi8q8ZY4pL8otSzgjLj6t'
]

In [ ]:
len(usdt_transfers)

In [ ]:
df = pd.DataFrame(usdt_transfers, columns=["amount", "date"])

In [ ]:
df['amount'].sum()

In [ ]:
fig = px.bar(df, x="date", y="amount", title="USDT transfers")
fig.update_layout(showlegend=False)
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='black')
fig.update_traces(marker_color=df['amount'].apply(lambda x: 'green' if x > 0 else 'red'))
fig.update_traces(width=500000000)
fig.show()

In [ ]:
def show_incomes_expenses(df, grouping_freq="M"):
    incomes = df[df['amount'] > 0]
    expenses = df[df['amount'] < 0]

    incomes_grouped = incomes.groupby(pd.Grouper(key="date", freq=grouping_freq)).sum().reset_index()
    expenses_grouped = expenses.groupby(pd.Grouper(key="date", freq=grouping_freq)).sum().reset_index()
    together = pd.concat([incomes_grouped, expenses_grouped], axis=0).sort_values("date")

    fig = px.bar(together, x="date", y="amount", title="USDT transfers")
    fig.update_layout(showlegend=False)
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='black')
    fig.update_traces(marker_color=together['amount'].apply(lambda x: 'green' if x > 0 else 'red'))
    fig.update_traces(width=500000000)
    fig.show()
    
    


In [ ]:
show_incomes_expenses(df, "M")